In [1]:
%matplotlib inline
import os
import json
import codecs
import numpy as np
import matplotlib.pyplot as plt
import openquake.hazardlib.geo.geodetic as geo
import hmtk.sources as src
import hmtk.plotting.seismicity.catalogue_plots as cp

from string import Template
from collections import OrderedDict
from hmtk.plotting.mapping import HMTKBaseMap
from hmtk.parsers.catalogue import CsvCatalogueParser
from hmtk.parsers.source_model.nrml04_parser import nrmlSourceModelParser
from hmtk.seismicity.selector import CatalogueSelector

In [2]:
# Alkan Hotel, Marmaris, Turkey
target_lon = 28.2520426
target_lat = 36.8429408

sources_file = "../geology/MarmarisNearbySources21.xml"
catalogue_filename = '../seismicity/output_homogenized_Marmaris.csv'

In [3]:
parser = CsvCatalogueParser(catalogue_filename) # From .csv to hmtk

# Read and process the catalogue content in a variable called "catalogue"
catalogue = parser.read_file() 

print 'Minimum magnitude: ', np.min(catalogue.data['magnitude'])
print 'Maximum magnitude: ', np.max(catalogue.data['magnitude'])
print 'Number of events: ', len(catalogue.data['magnitude'])
print 'Catalogue keys: '
print catalogue.data.keys()

Catalogue Attribute  is not a recognised catalogue key
Catalogue Attribute Identifier is not a recognised catalogue key
Minimum magnitude:  0.75
Maximum magnitude:  7.6
Number of events:  30006
Catalogue keys: 
['eventID', 'comment', 'sigmaMagnitude', 'hour', 'SemiMinor90', 'magnitude', 'Agency', 'second', 'longitude', 'month', 'depthError', 'flag', 'depth', 'magnitudeType', 'SemiMajor90', 'timeError', 'year', 'latitude', 'ErrorStrike', 'day', 'minute']


In [4]:
cp.plot_depth_histogram(catalogue, 10)

IndexError: index out of bounds

In [ ]:
cp.plot_magnitude_time_density(catalogue, 0.2, 2)

In [ ]:
completeness_table_a = np.array([[1992, 3.5],
                                 [1978, 4.0], 
                                 [1963, 5.0],
                                 [1900, 7.0]]) 
cp.plot_observed_recurrence(catalogue, completeness_table_a, 0.1)

In [ ]:
# Map configuration
span = 1 # deg
map_config = {"min_lon": target_lon - 2*span, 
              "max_lon": target_lon + 2*span, 
              "min_lat": target_lat - span, 
              "max_lat": target_lat + span, "resolution": "l"}

In [ ]:
# Creating a basemap 
basemap1 = HMTKBaseMap(map_config, 'Catalogue and nearest sources')

# Adding the catalogue to the basemap
basemap1.add_catalogue(catalogue, overlay=True)

# Reading the models 
parser = nrmlSourceModelParser(sources_file)

# Parse the seismic sources and save them in "source_model"
source_model = parser.read_file("Sources Around Marmaris")

# Adding the seismic sources
basemap1.add_source_model(source_model, area_border='r-', 
                          border_width=1.5, alpha=0.5, overlay=True)

# Add target
basemap1.add_size_scaled_points(target_lon, target_lat, 20, shape='*', 
                                colour='k', zorder=6)

In [ ]:
# Creating a basemap 
basemap1 = HMTKBaseMap(map_config, 'Catalogue and "most dangerous" sources')

# Adding the catalogue to the basemap
basemap1.add_catalogue(catalogue, overlay=True)

# Reading the models 
parser = nrmlSourceModelParser(dangerous_sources_file)

# Parse the seismic sources and save them in "source_model"
source_model = parser.read_file("Sources Around Marmaris")

# Adding the seismic sources
basemap1.add_source_model(source_model, area_border='r-', 
                          border_width=1.5, alpha=0.5) 

In [ ]:
selector = CatalogueSelector(catalogue, create_copy=True)

In [ ]:
for source in source_model.sources:
    if isinstance(source, src.area_source.mtkAreaSource): 
        source.select_catalogue(selector)
        print 'Area source %s, name %s, # of events %8.0f' % (
            source.id, source.name, source.catalogue.get_number_events())
        #subcatalogue_area = selector.    

In [ ]:
d_km = 10*np.ones_like(source_model.sources)
d_km[-1] = 30
for i, source in enumerate(source_model.sources):
    if isinstance(source, src.simple_fault_source.mtkSimpleFaultSource): 
        source.select_catalogue(selector, d_km[i])
    elif isinstance(source, src.complex_fault_source.mtkComplexFaultSource): 
        source.select_catalogue(selector, d_km[i]) 
        
    print '%s: %s, %d events wihin %g km' % (
        source.id, source.name, source.catalogue.get_number_events(), d_km[i])

In [ ]:
for i, source in enumerate(source_model.sources):
    n_events = source.catalogue.get_number_events()
    if n_events < 5: continue
    basemap = HMTKBaseMap(
        map_config, '%d events %d km from %s' % (n_events, d_km[i], source.name))
    basemap.add_catalogue(source.catalogue, overlay=True)
    basemap.add_source_model(source_model, area_border='k-')
    
    cp.plot_observed_recurrence(source.catalogue, completeness_table_a, 0.1)